In [1]:
importall Base

In [2]:
include("frames.jl");

In [3]:
abstract CacheElement{T}

type ImmutableCacheElement{T} <: CacheElement{T}
    data::T
end

get{T}(element::ImmutableCacheElement{T}) = element.data
function setdirty!{T}(element::ImmutableCacheElement{T})
end

type MutableCacheElement{T} <: CacheElement{T}
    data::T
    updateFunction::Function
    dirty::Bool
    
    MutableCacheElement(data::T, updateFunction::Function) = new(data, updateFunction, true)
end
MutableCacheElement{T}(data::T, updateFunction::Function) = MutableCacheElement{T}(data, updateFunction)

function get{T}(element::MutableCacheElement{T})
    if element.dirty
        element.data = element.updateFunction()
        element.dirty = false
    end
    return element.data
end
function setdirty!{T}(element::MutableCacheElement{T})
    element.dirty = true
end

setdirty! (generic function with 2 methods)

In [4]:
type FrameCache{T}
    root::CartesianFrame3D
    transformsToParent::Dict{CartesianFrame3D, CacheElement{Transform3D{T}}}
    transformsToWorld::Dict{CartesianFrame3D, CacheElement{Transform3D{T}}}
    dependencies::Dict{CacheElement{Transform3D{T}}, Vector{CacheElement{Transform3D{T}}}}
    
    function FrameCache(root::CartesianFrame3D)
        transformsToParent = Dict{CartesianFrame3D, CacheElement{Transform3D{T}}}()
        transformsToWorld = Dict{CartesianFrame3D, CacheElement{Transform3D{T}}}(root => ImmutableCacheElement(Transform3D{T}(root, root)))
        dependencies = Dict{CacheElement{Transform3D{T}}, Vector{CacheElement{Transform3D{T}}}}()
        new(root, transformsToParent, transformsToWorld, dependencies)
    end
end

In [11]:
function add_frame!{T}(cache::FrameCache{T}, t::Transform3D{T})
    # for fixed frames
    parent_to_world = cache.transformsToWorld[t.to]

    to_parent = ImmutableCacheElement(t)
    cache.transformsToParent[t.from] = to_parent

    update_fun = () -> begin println("updating"); get(parent_to_world) * get(to_parent) end
    to_world = MutableCacheElement(Transform3D{T}(t.from, root), update_fun)
    cache.transformsToWorld[t.from] = to_world
    
    cache.dependencies[to_world] = [parent_to_world]
    return cache
end

function setdirty!{T}(cache::FrameCache{T}, element::CacheElement{Transform3D{T}})
    setdirty!(element)
    for dependent in cache.dependencies
        setdirty!(cache, element)
    end
end

function setdirty!{T}(cache::FrameCache{T})
    for element in values(cache.transformsToParent)
        setdirty!(element)
    end
end

function transform_to_parent{T}(cache::FrameCache{T}, frame::CartesianFrame3D)
    frame == cache.root && error("frame has no parent")
    return get(cache.transformsToParent[frame])
end

transform_to_world{T}(cache::FrameCache{T}, frame::CartesianFrame3D) = get(cache.transformsToWorld[frame])

function relative_transform{T}(cache::FrameCache{T}, from::CartesianFrame3D, to::CartesianFrame3D)
    return inv(transform_to_world(cache, to)) * transform_to_world(cache, from)
end

relative_transform (generic function with 1 method)

In [12]:
root = CartesianFrame3D("root")
cache = FrameCache{Float64}(root)
f1 = CartesianFrame3D("f1")
t1 = Transform3D(f1, root, nquatrand(), rand(Vec{3, Float64}))
add_frame!(cache, t1)

FrameCache{Float64}(TreeVertex{CartesianFrame3D,CacheElement{Transform3D{Float64}}}(CartesianFrame3D("root"),Nullable{TreeVertex{CartesianFrame3D,CacheElement{Transform3D{Float64}}}}(),Nullable{CacheElement{Transform3D{Float64}}}(),[TreeVertex{CartesianFrame3D,CacheElement{Transform3D{Float64}}}(CartesianFrame3D("f1"),Nullable(TreeVertex{CartesianFrame3D,CacheElement{Transform3D{Float64}}}(#= circular reference =#)),Nullable(ImmutableCacheElement{Transform3D{Float64}}(Transform3D{Float64}(CartesianFrame3D("f1"),CartesianFrame3D("root"),0.6715146146452259 - 0.4649069963259432im + 0.5140255924336238jm - 0.26212076874663176km,FixedSizeArrays.Vec{3,Float64}((0.8065669931258241,0.37159043518196366,0.6242239048416662))))),TreeVertex{CartesianFrame3D,CacheElement{Transform3D{Float64}}}[])]),Dict{CartesianFrame3D,CacheElement{Transform3D{Float64}}}(CartesianFrame3D("root")=>ImmutableCacheElement{Transform3D{Float64}}(Transform3D{Float64}(CartesianFrame3D("root"),CartesianFrame3D("root"),1.0 + 

In [13]:
transform_to_parent(cache, f1)

Transform3D{Float64}(CartesianFrame3D("f1"),CartesianFrame3D("root"),0.6715146146452259 - 0.4649069963259432im + 0.5140255924336238jm - 0.26212076874663176km,FixedSizeArrays.Vec{3,Float64}((0.8065669931258241,0.37159043518196366,0.6242239048416662)))

In [14]:
transform_to_world(cache, f1)

updating


Transform3D{Float64}(CartesianFrame3D("f1"),CartesianFrame3D("root"),0.6715146146452259 - 0.4649069963259432im + 0.5140255924336238jm - 0.26212076874663176km,FixedSizeArrays.Vec{3,Float64}((0.8065669931258241,0.37159043518196366,0.6242239048416662)))

In [17]:
relative_transform(cache, root, f1)

Transform3D{Float64}(CartesianFrame3D("root"),CartesianFrame3D("f1"),0.671514614645226 + 0.4649069963259433im - 0.5140255924336239jm + 0.2621207687466318km,FixedSizeArrays.Vec{3,Float64}((0.31770297485671434,0.4996253208511252,-0.9097937171832616)))

In [16]:
setdirty!(cache)

TreeVertex{CartesianFrame3D,CacheElement{Transform3D{Float64}}}(CartesianFrame3D("root"),Nullable{TreeVertex{CartesianFrame3D,CacheElement{Transform3D{Float64}}}}(),Nullable{CacheElement{Transform3D{Float64}}}(),[TreeVertex{CartesianFrame3D,CacheElement{Transform3D{Float64}}}(CartesianFrame3D("f1"),Nullable(TreeVertex{CartesianFrame3D,CacheElement{Transform3D{Float64}}}(#= circular reference =#)),Nullable(ImmutableCacheElement{Transform3D{Float64}}(Transform3D{Float64}(CartesianFrame3D("f1"),CartesianFrame3D("root"),0.6715146146452259 - 0.4649069963259432im + 0.5140255924336238jm - 0.26212076874663176km,FixedSizeArrays.Vec{3,Float64}((0.8065669931258241,0.37159043518196366,0.6242239048416662))))),TreeVertex{CartesianFrame3D,CacheElement{Transform3D{Float64}}}[])])

In [ ]:
# J = rand(Mat{3, 3, Float64}); J = J * J'
# c = rand(Vec{3, Float64})
# m = rand()
# inertia = SpatialInertia(f3, J, c, m)
# m = Mechanism()
# world = add_body!(m, RigidBody("world"))
# body = add_body!(m, RigidBody("body", inertia))
# add_joint!(m, QuaternionFloatingJoint("joint"), world, body)

In [ ]:
# transform(inertia, t3)